![calculate_logo](https://raw.githubusercontent.com/newlawrence/Talks/master/180726_calculate/pictures/calculate.svg?sanitize=true)

## Alberto Lorenzo Márquez

* Ingeniero aeroespacial

* Desarrollador de software (**C++**, **Fortran**, **Python**, **Javascript**)

* **Github**: https://github.com/newlawrence

* **Twitter**: [@newlawrence](https://twitter.com/newlawrence?lang=es)

# Uso básico de la biblioteca

In [1]:
#pragma cling add_include_path("calculate2.1.1rc5")

In [2]:
#include "calculate.hpp"

## Instanciar un parser

In [3]:
auto parser = calculate::Parser{};

## Crear expresiones atendiendo a su notación

**Calculate** permite parsear expresiones escritas tanto en notación infija como posfija:

In [4]:
double result;

In [5]:
auto e1 = parser.from_infix("1+2*3");
result = e1;
result

7

In [6]:
auto e2 = parser.from_postfix("1 2 3 * +");
result = e2;
result

7

## Crear expresiones con variables

Aquellos símbolos que no se encuentren cargados en un parser harán saltar una excepción:

In [7]:
#include <iostream>

In [8]:
try {
    parser.from_infix("x");
}
catch (const calculate::BaseError& error) {
    std::cout << error.what() << std::endl;
}

Undefined symbol: 'x'


Es posible especificar qué símbolos se han de tratar como variables al momento de construir un objeto expresión. Las expresiones se evalúan como si de funciones regulares se tratase.

In [9]:
auto e3 = parser.from_infix("x+y", "x", "y");
e3(1, 2)  // x == 1, y == 2

3

In [10]:
auto e4 = parser.from_postfix("x y +", "x", "y");
e4(1, 2)  // x == 1, y == 2

3

Los objetos de la clase **Parser** proveen además de un método adicional **parse** que va añadiendo a la lista de variables aquellos símbolos que no están previamente cargados según van apareciendo en la expresión:

In [11]:
auto e5 = parser.parse("a-b");

for (const auto& variable : e5.variables())
    std::cout << variable << " ";
std::cout << std::endl;

e5(1, 2)  // a == 1, b == 2

a b 


-1

# La clase Parser

* Provee de los métodos para crear expresiones.

* Permite generar las expresiones precalculando las ramas constantes de las mismas.

* Dispone de unos contenedores para los distintos tipos de símbolos: **constantes**, **operadores** y **funciones**.

## Optimizar una expresión

In [12]:
parser.parse("1+2+x").infix()

"1+2+x"

Para generar expresiones precalculando aquellas ramas constantes se modifica el atributo optimize del objeto **parser**:

In [13]:
parser.optimize = true;
parser.parse("1+2+x").infix()

"3+x"

### ¡Atención!

**Calculate** no dispone de capacidades **CAS**, sólo puede optimizar aquellas ramas para el árbol que ella misma genera. No es capaz de transformar dicho árbol a una forma canónica para llevar a cabo optimizaciones más evidentes desde el punto de vista matemático.

In [14]:
parser.parse("1+x+2").infix()

"1+x+2"

In [15]:
parser.optimize = false;

## Contenedores de símbolos

Los objetos de la clase **Parser** tienen contenedores específicos para:

In [16]:
{ parser.constants; }  // Constantes

In [17]:
{ parser.functions; }  // Funciones

In [18]:
{ parser.operators; }  // Operadores

Los contenedores proveen una interfaz equivalente a la de la clase **unordered_map** de la **STL** (con algunas licencias en cuanto a la implementación):

In [19]:
result = parser.constants["pi"];
result

3.14159

In [20]:
for (const auto& operator_pair : parser.operators)
    std::cout << operator_pair.first << "   ";
std::cout << std::endl;

^   %   *   -   /   +   


Por ejemplo, añadir una constante es tan sencillo como:

In [21]:
{ parser.constants.insert({"G", {6.674e-11}}); }   // Gravitación universal

In [22]:
auto f_grav = parser.from_infix("G * m1 * m2 / r^2", "m1", "m2", "r");
f_grav(1, 5.972e24, 6371e3) // Gravedad terrestre en superficie

9.81953

# Las subclases de la clase Símbolo

Como ya se vio, las expresiones pueden estar formadas por tres tipos de símbolos diferentes (sin contar los paréntesis y la coma de separación propios de la notación posfija):

* Constantes

* Funciones

* Operadores

*Existe un cuarto tipo, las **Variables**, pero estas pertenecen al funcionamiento interno del parser y sólo existen para proporcionar la habilidad a las expresiones de funcionar como invocables. A efectos prácticos, funcionan como las constantes.*

## Constantes

Envuelven valores del tipo al que esté orientado el parser (**double** en el caso del parser por defecto).

In [23]:
calculate::Parser::Constant c{1};
double{c}

1

## Funciones

Son capaces de envolver cualquier tipo de invocable, siempre que todos sus argumentos (un número indefinido de ellos) y el resultado sean del tipo al que está orientado el parser:

In [24]:
double sum(double x, double y) { return x + y; }

In [25]:
calculate::Parser::Function f{sum};
f(2, 2)

4

En el caso de funtores, estos han de tener su operador llamada declarado como constante (las funciones matemáticas no tienen efectos secundarios).

In [26]:
class Plus {
    double _x;

public:
    Plus(double x) : _x{x} {}
    double operator()(double x) const { return x + _x; }
};

In [27]:
f = Plus(2);
f(2)

4

Dado que el número de argumentos del operador llamada de los objetos de la clase **Function** es indefinido, los errores en el número de ellos pasan al tiempo de ejecución:

In [28]:
try {
    f(2, 2);
}
catch (const calculate::BaseError& error) {
    std::cout << error.what() << std::endl;
}

Arguments mismatch: 1 needed argument vs 2 provided


## Operadores

Al igual que las funciones, envuelven invocables, pero además poseen dos atributos adicionales:

* Su precedencia (dada por un entero sin signo).

* Su asociatividad (por la izquierda, por la derecha o ambas).

Por ejemplo, el operador **+** por defecto:

In [29]:
calculate::Parser::Operator plus = parser.operators["+"];

Tiene menor precedencia que el operador **\*** por defecto:

In [30]:
bool{plus.precedence() < parser.operators["*"].precedence()}

true

Y es asociativo por ambos lados:

$(1 + 2) + 3 == 1 + (2 + 3)$

In [31]:
bool{plus.associativity() == calculate::Parser::Associativity::FULL}

true